In [16]:
import math
import sys
import os
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.table import Table
import matplotlib
from matplotlib import pyplot as plt
import time


sys.path.insert(0, "/global/homes/c/cinlima/gcrcatalogs-new/gcr-catalogs")
sys.path.insert(0, "global/homes/c/cinlima/clevar/clevar")

import GCRCatalogs
#Put nersc if using nersc
GCRCatalogs.set_root_dir_by_site("nersc")
print('GCRCatalogs =', GCRCatalogs.__version__, '|' ,'GCR =', GCRCatalogs.GCR.__version__)

from clevar import ClCatalog, MemCatalog
from clevar.match import MembershipMatch, get_matched_pairs

from clevar.match import output_matched_catalog
import clevar

print('Clevar =', clevar.__version__)


# matplotlib.rcParams.update({'font.size': 22})
# matplotlib.rcParams.update({'figure.figsize': (10,8)})


GCRCatalogs = 1.9.0 | GCR = 0.9.2
Clevar = 0.14.1


In [2]:
gc = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_redmapper_v0.8.1')
gc_truth = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image')  

In [3]:
quantities = gc.list_all_quantities()
cluster_quantities = [q for q in quantities if 'member' not in q]
member_quantities = [q for q in quantities if 'member' in q]

In [4]:
# query = GCRCatalogs.GCRQuery('(richness > ' + str(min_richness) +')')   
cluster_data = Table(gc.get_quantities(cluster_quantities))
member_data = Table(gc.get_quantities(member_quantities))

In [21]:
# cluster_data[cluster_data['richness'] < 5]

In [14]:
# min_halo_mass = 1e13 #Msun
# redshift_max = 1.2

# quantities_wanted = ['redshift','halo_mass','halo_id','galaxy_id','ra','dec', 'is_central', 'mag_true_u_lsst', 'mag_true_g_lsst', 'mag_true_r_lsst', 'mag_true_i_lsst', 'mag_true_z_lsst', 'mag_true_y_lsst']

# query = GCRCatalogs.GCRQuery('(halo_mass > ' + str(min_halo_mass) +') & (redshift < ' + str(redshift_max) +')')
# truth_data = Table(gc_truth.get_quantities(quantities_wanted, [query]))
truth_data = Table.read('cosmoDC2_image_selected_data.fits')


In [18]:
RM_cat_name = 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
DC2_cat_name = 'cosmoDC2_v1.1.4_image'

#Redmapper
RM_cat = ClCatalog(RM_cat_name, z=cluster_data["redshift"], ra=cluster_data["ra"], dec=cluster_data["dec"], id = cluster_data["cluster_id"], richness = cluster_data["richness"], richness_err=cluster_data["richness_err"], \
                   id_cg = cluster_data["id_cen_0"], ztrue_cg=cluster_data["redshift_true_cg"], p_cg_0=cluster_data["p_cen_0"])

pmem = member_data["p_member"] * member_data["pfree_member"] * member_data["theta_i_member"] * member_data["theta_r_member"] 

member_data.add_column(pmem, name="pmem")

RM_cat_members = MemCatalog("RM_members", id_cluster=member_data["cluster_id_member"], id=member_data["id_member"], ra=member_data["ra_member"], dec=member_data["dec_member"],
                            pmem=member_data["pmem"],  redshift=member_data["redshift_true_member"], 
                            mag_g_lsst_member=member_data["mag_g_lsst_member"], mag_r_lsst_member=member_data["mag_r_lsst_member"], 
                            mag_i_lsst_member=member_data["mag_i_lsst_member"], mag_z_lsst_member=member_data["mag_z_lsst_member"], mag_y_lsst_member=member_data["mag_y_lsst_member"]) 

RM_cat.add_members(members_catalog=RM_cat_members)


#Dc2
halo_cat = ClCatalog(DC2_cat_name, z= truth_data["redshift"][truth_data["is_central"]],
                     ra= truth_data["ra"][truth_data["is_central"]], dec= truth_data["dec"][truth_data["is_central"]],
                     id=truth_data["halo_id"][truth_data["is_central"]], mass_fof=truth_data["halo_mass"][truth_data["is_central"]])

halo_cat_members = MemCatalog("halo_members", id_cluster=truth_data["halo_id"], id=truth_data["galaxy_id"], ra=truth_data["ra"], dec=truth_data["dec"],
                              redshift=truth_data["redshift"], is_central = truth_data["is_central"], 
                              mag_true_u_lsst = truth_data["mag_true_u_lsst"],  mag_true_g_lsst = truth_data["mag_true_g_lsst"],
                              mag_true_r_lsst = truth_data["mag_true_r_lsst"], mag_true_i_lsst = truth_data["mag_true_i_lsst"],  mag_true_z_lsst = truth_data["mag_true_z_lsst"], ) 

halo_cat.add_members(members_catalog=halo_cat_members)

RM_cat._init_match_vals(True)
halo_cat._init_match_vals(True)

ValueError: Cannot replace column 'pmem'.  Use Table.replace_column() instead.

In [ ]:
# RM_cat_name = 'skysim5000_v1.1.1_redmapper_v0.8.5'
# DC2_cat_name = 'skysim5000_v1.1.1_image'

RM_cat_name = 'cosmoDC2_v1.1.4_redmapper_v0.8.1'
DC2_cat_name = 'cosmoDC2_v1.1.4_image'

RM_cat = ClCatalog(RM_cat_name, z=cluster_data["redshift"], ra=cluster_data["ra"], dec=cluster_data["dec"], id = cluster_data["cluster_id"], richness = cluster_data["richness"], richness_err=cluster_data["richness_err"], \
                   id_cg = cluster_data["id_cen_0"], ztrue_cg=cluster_data["redshift_true_cg"], p_cg_0=cluster_data["p_cen_0"])



In [ ]:
display(RM_cat)

In [ ]:
#RM_cat.write('RM_cat.fits', add_header =True, overwrite=True)


In [ ]:
pmem = member_data["p_member"] * member_data["pfree_member"] * member_data["theta_i_member"] * member_data["theta_r_member"] 

member_data.add_column(pmem, name="pmem")

In [ ]:
RM_cat_members = MemCatalog("RM_members", id_cluster=member_data["cluster_id_member"], id=member_data["id_member"], ra=member_data["ra_member"], dec=member_data["dec_member"],
                            pmem=member_data["pmem"],  redshift=member_data["redshift_true_member"], 
                            mag_g_lsst_member=member_data["mag_g_lsst_member"], mag_r_lsst_member=member_data["mag_r_lsst_member"], 
                            mag_i_lsst_member=member_data["mag_i_lsst_member"], mag_z_lsst_member=member_data["mag_z_lsst_member"], mag_y_lsst_member=member_data["mag_y_lsst_member"]) 
RM_cat.add_members(members_catalog=RM_cat_members)

# RM_cat.members.write('RM_cat_members.fits', overwrite=True)


In [ ]:
halo_cat = ClCatalog(DC2_cat_name, z= truth_data["redshift"][truth_data["is_central"]],
                     ra= truth_data["ra"][truth_data["is_central"]], dec= truth_data["dec"][truth_data["is_central"]],
                     id=truth_data["halo_id"][truth_data["is_central"]], mass_fof=truth_data["halo_mass"][truth_data["is_central"]])
halo_cat

In [ ]:
# halo_cat.write('halo_cat.fits', overwrite=True)


In [ ]:
halo_cat_members = MemCatalog("halo_members", id_cluster=truth_data["halo_id"], id=truth_data["galaxy_id"], ra=truth_data["ra"], dec=truth_data["dec"],
                              redshift=truth_data["redshift"], is_central = truth_data["is_central"], 
                              mag_true_u_lsst = truth_data["mag_true_u_lsst"],  mag_true_g_lsst = truth_data["mag_true_g_lsst"],
                              mag_true_r_lsst = truth_data["mag_true_r_lsst"], mag_true_i_lsst = truth_data["mag_true_i_lsst"],  mag_true_z_lsst = truth_data["mag_true_z_lsst"], ) 


In [ ]:
halo_cat_members

In [ ]:
halo_cat.add_members(members_catalog=halo_cat_members)


In [ ]:
# halo_cat.members.write('halo_cat_members.fits', overwrite=True)


In [ ]:
# RM_cat = ClCatalog.read('RM_cat_richness_min_'+str(min_richness)+'.fits', name='RM_cat', full=True, tags={'mass':'richness'})
# halo_cat = ClCatalog.read('halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'.fits', name='halo_cat', full=True, tags={'mass':'mass_fof'})



In [ ]:
# RM_cat.read_members('RM_cat_richness_min_'+str(min_richness)+'_members.fits',  full=True)
# halo_cat.read_members('halo_cat_mass_min_'+f"{min_halo_mass:1.2e}"+'_members.fits', full=True)


In [ ]:
RM_cat._init_match_vals(True)
halo_cat._init_match_vals(True)

In [ ]:
mt = MembershipMatch()
match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'preference': 'shared_member_fraction', # other options are more_massive, angular_proximity or redshift_proximity
    'minimum_share_fraction': 0,
    'match_members_kwargs': {'method':'id'},
}
mt.match_from_config(RM_cat, halo_cat, match_config)


In [ ]:
# mt.save_matches(RM_cat, halo_cat, out_dir='cDC2_RM_match_cross_membership', overwrite=True)


In [ ]:
# mt.load_matches(RM_cat, halo_cat, out_dir='cDC2_RM_match_cross_membership')


In [ ]:
RM_cat_bij, halo_cat_bij = get_matched_pairs(RM_cat, halo_cat, 'cross')

In [ ]:
RM_cat_bij.size, RM_cat_bij.size/RM_cat.size,  halo_cat_bij.size/halo_cat.size


In [ ]:
plt.scatter(np.log(halo_cat_bij['mass_fof']),np.log(RM_cat_bij['richness']))
# plt.scatter(halo_cat_bij['mass_fof'],RM_cat_bij['richness'])

plt.xlabel('$\ln (M_{fof}$)')
plt.ylabel('$\ln (\lambda)$')

In [ ]:
Table([halo_cat_bij['mass_fof'], RM_cat_bij['richness'], RM_cat_bij['z']])